In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
#raw_tables
prior = pd.read_csv("prior.csv")

con = sqlite3.connect("myDB.db")
user = pd.read_sql_query("SELECT * from user", con)
answer = pd.read_sql_query("SELECT * from answers", con)
con.close()

user['time_hash'] = user['time_hash'] .astype(int)
answer['task_time'] = (pd.to_datetime(answer.end_time) - pd.to_datetime(answer.start_time))/np.timedelta64(1,'s')

data = pd.merge(user,answer,left_on='time_hash',right_on='user_hid')
numbers = data[['id_x','username','group','block_num','task_num','answer','task_time']].dropna()

In [3]:
#task_tables
def table_maker(tn4bl1, tn4bl2, check_string):
    bl0t = numbers.loc[(numbers.block_num == 0) & (numbers.task_num ==tn4bl1), 
                       ['id_x','username','group','answer', 'task_time']]
    bl1t = numbers.loc[(numbers.block_num == 1) & (numbers.task_num ==tn4bl2), 
                       ['id_x','username','group','answer', 'task_time']]
    
    table_task = pd.merge(bl0t,bl1t,on=['id_x', 'username','group'])
    table_task['delta_time'] = table_task.task_time_y-table_task.task_time_x
    table_task['change'] = (table_task.answer_x.str.contains(check_string, regex=False)!=
                             table_task.answer_y.str.contains(check_string, regex=False)) 
    table_task.loc[table_task.answer_x.str.contains(check_string, regex=False), 'change'] = 'No_set'
    table_task = pd.merge(prior,table_task, on='username')
    table_task.loc[~(table_task.set.str.contains(check_string, regex=False)), 'change'] = 'No_preset'
    return table_task

lambda_table = table_maker(3, 2, check_string ='lambda')
isinstance_table = table_maker(0, 1, check_string ='isinstance')
enumerate_table = table_maker(1, 0, check_string ='enumerate')
sum_table = table_maker(2, 3, check_string ='sum(')

In [4]:
#analysis_tables
def no_preset_table(table_name):
    no_preset_table = table_name.loc[table_name.change.str.contains('No_preset',regex=False), 
                                     ['id_x', 'group', 'answer_x', 'answer_y', 'delta_time']]
    return no_preset_table

def no_set_table(table_name):
    no_set_table = table_name.loc[table_name.change.str.contains('No_set',regex=False), 
                                     ['id_x', 'group', 'answer_x', 'answer_y', 'delta_time']]
    return no_set_table

def is_change_table(table_name):
    is_change_table = table_name.loc[(table_name.change.isin([True, False])), 
                                  ['id_x', 'group', 'answer_x', 'answer_y', 'delta_time', 'change']]
    return is_change_table